
# Sistema de Gerenciamento de Estoque de Farmácia

Este notebook implementa um sistema de gerenciamento de estoque para uma farmácia, desenvolvido em Python com uma abordagem orientada a objetos. O sistema permite realizar operações CRUD (Criar, Ler, Atualizar, Deletar) para medicamentos, processar pedidos de clientes e exibir resumos do estoque.

Funcionalidades Adicionais Implementadas:

Interface de menu de usuário via console.
 - Controle de necessidade de receita para medicamentos.
 - Classificação de medicamentos como genéricos ou de marca.
 - Atribuição de ID numérico único e crescente para cada medicamento.
 - Uso de cores no texto do console para melhorar a experiência do usuário (UX).
 - Diversas validações para prevenir erros de entrada do usuário.
 - Funcionalidade de desconto por CPF no processamento de pedidos.
 - Aviso dinâmico para medicamentos em estoque crítico ou esgotado.

Foram adicionados previamente 5 medicamentos à memória para facilitar a visualização e teste do programa.



### 1. Configurações Iniciais e Definições Globais
Este bloco de código estabelece as variáveis globais para o controle do estoque e IDs de medicamentos, define o limite para estoque crítico e introduz a classe Cores para estilização do output no console, melhorando a interface com o usuário.


In [12]:
# Variáveis globais e constantes
estoque = {}
proximo_id_medicamento = 1
LIMITE_ESTOQUE_CRITICO = 5

class Cores:
    RESET = '\033[0m'
    AZUL = '\033[94m'
    VERDE = '\033[92m'
    AMARELO = '\033[93m'
    VERMELHO = '\033[91m'
    MAGENTA = '\033[95m'
    CIANO = '\033[96m'
    BRANCO = '\033[97m'

    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

### 2. Definição da Classe Medicamento
A classe Medicamento encapsula os atributos de um medicamento, como nome, preço, necessidade de receita, se é genérico, quantidade em estoque e um ID único. Esta classe serve como modelo para todos os medicamentos gerenciados pelo sistema.



In [13]:
class Medicamento:
    def __init__(self, nome, preco, receita, generico, quantidade=0, id=None):
        """Inicializa um novo objeto Medicamento."""
        self.nome = nome
        self.preco = preco
        self.receita = receita
        self.generico = generico
        self.quantidade = quantidade
        self.id = id

### 3. Funções de Gerenciamento de Estoque
Este conjunto de funções constitui o núcleo das operações de gerenciamento de estoque. Inclui funcionalidades para listar todos os medicamentos, adicionar novos medicamentos com validação de dados, atualizar a quantidade de medicamentos existentes e remover medicamentos do estoque.



In [14]:
def listar_estoque():
    """
    Lista todos os medicamentos no estoque, numerados, com seus atributos
    formatados e espaçados, incluindo avisos de estoque crítico/esgotado.
    """
    print(f"\n{Cores.BOLD}{Cores.CIANO}--- LISTA COMPLETA DO ESTOQUE ---{Cores.RESET}\n")
    if not estoque:
        print(f"{Cores.AMARELO}Estoque vazio. Não há medicamentos para listar.{Cores.RESET}")
        return

    numero_item = 1
    for medicamento_obj in estoque.values():
        if medicamento_obj.quantidade == 0:
            nome_display = f"{Cores.BOLD}{Cores.VERMELHO}{medicamento_obj.nome.upper()}      ESGOTADO{Cores.RESET}"
        elif medicamento_obj.quantidade > 0 and medicamento_obj.quantidade <= LIMITE_ESTOQUE_CRITICO:
            nome_display = f"{Cores.VERMELHO}{medicamento_obj.nome.upper()}      ESTOQUE CRÍTICO{Cores.RESET}"
        else:
            nome_display = f"{Cores.AMARELO}{medicamento_obj.nome.upper()}{Cores.RESET}"

        print(f"{numero_item}. {nome_display}")
        print(f"      Preço: R${medicamento_obj.preco:.2f}")
        cor_qtd_listar = Cores.VERMELHO if medicamento_obj.quantidade <= LIMITE_ESTOQUE_CRITICO else Cores.VERDE
        print(f"      Quantidade: {cor_qtd_listar}{medicamento_obj.quantidade}{Cores.RESET}")
        print(f"      Exige Receita: {'Sim' if medicamento_obj.receita else 'Não'}")
        print(f"      É Genérico: {'Sim' if medicamento_obj.generico else 'Não'}")
        print(f"      ID Único: {medicamento_obj.id}")
        print(f"{Cores.CIANO}---------------------------------{Cores.RESET}")
        numero_item += 1
    print(f"{Cores.BOLD}{Cores.CIANO}--- FIM DA LISTA DE ESTOQUE ---{Cores.RESET}")

def adicionar_medicamento():
    """
    Permite adicionar um novo medicamento ao estoque, solicitando
    seus atributos e validando as entradas do usuário.
    """
    global proximo_id_medicamento
    print(f"\n{Cores.BOLD}{Cores.VERDE}--- ADICIONAR NOVO MEDICAMENTO ---{Cores.RESET}")

    nome_medicamento = input("Digite o nome do medicamento (ou digite '0' para cancelar): ").strip()
    if nome_medicamento == '0':
        print(f"{Cores.AZUL}Operação de adição de medicamento cancelada.{Cores.RESET}")
        return
    if not nome_medicamento:
        print(f"{Cores.VERMELHO}Nome do medicamento não pode ser vazio.{Cores.RESET}")
        return

    while True:
        try:
            print(f"Digite o preço de {Cores.AMARELO}{nome_medicamento}{Cores.RESET} (ex: 15.75): ", end="")
            preco_str = input().strip()
            preco_medicamento = float(preco_str)
            if preco_medicamento < 0:
                print(f"{Cores.VERMELHO}O preço não pode ser negativo. Tente novamente.{Cores.RESET}")
                continue
            break
        except ValueError:
            print(f"{Cores.VERMELHO}Preço inválido. Por favor, digite um número.{Cores.RESET}")

    while True:
        print(f"Exige receita médica para {Cores.AMARELO}{nome_medicamento}{Cores.RESET}? (s/n): ", end="")
        receita_str = input().lower().strip()
        if receita_str == 's':
            receita_medicamento = True
            break
        elif receita_str == 'n':
            receita_medicamento = False
            break
        else:
            print(f"{Cores.VERMELHO}Resposta inválida. Por favor, digite 's' para sim ou 'n' para não.{Cores.RESET}")

    while True:
        generico_str = input(f"É um medicamento genérico? (s/n): ").lower().strip()
        if generico_str == 's':
            generico_medicamento = True
            break
        elif generico_str == 'n':
            generico_medicamento = False
            break
        else:
            print(f"{Cores.VERMELHO}Resposta inválida. Por favor, digite 's' para sim ou 'n' para não.{Cores.RESET}")

    while True:
        try:
            print(f"Digite a quantidade inicial de {Cores.AMARELO}{nome_medicamento}{Cores.RESET}: ", end="")
            quantidade_str = input().strip()
            quantidade_medicamento = int(quantidade_str)
            if quantidade_medicamento < 0:
                print(f"{Cores.VERMELHO}A quantidade inicial não pode ser negativa. Tente novamente.{Cores.RESET}")
                continue
            break
        except ValueError:
            print(f"{Cores.VERMELHO}Quantidade inválida. Por favor, digite um número inteiro.{Cores.RESET}")

    try:
        novo_medicamento_obj = Medicamento(nome_medicamento, preco_medicamento, receita_medicamento, generico_medicamento, quantidade_medicamento, proximo_id_medicamento)
        estoque[proximo_id_medicamento] = novo_medicamento_obj
        print(f"\nMedicamento {Cores.AMARELO}'{nome_medicamento}'{Cores.RESET} adicionado com {Cores.VERDE} ID {proximo_id_medicamento} {Cores.RESET}e {Cores.MAGENTA}{quantidade_medicamento} unidades.{Cores.RESET}")
        proximo_id_medicamento += 1
    except ValueError as e:
        print(f"{Cores.VERMELHO}Erro ao criar o medicamento: {e}. Não foi adicionado ao estoque.{Cores.RESET}")

def atualizar_estoque():
    """
    Permite ao usuário adicionar uma quantidade a um medicamento existente
    escolhendo-o por ID. A quantidade fornecida será SOMADA à quantidade atual.
    """
    print(f"\n{Cores.BOLD}{Cores.AZUL}--- ADICIONAR QUANTIDADE AO ESTOQUE POR ID ---{Cores.RESET}")
    if not estoque:
        print(f"{Cores.AMARELO}Estoque vazio. Não há medicamentos para adicionar quantidade.{Cores.RESET}")
        return

    print("Medicamentos disponíveis para adicionar quantidade:")
    for id_medicamento, medicamento_obj in estoque.items():
        if medicamento_obj.quantidade <= LIMITE_ESTOQUE_CRITICO:
            print(f"      {id_medicamento}. {Cores.AMARELO}{medicamento_obj.nome}{Cores.RESET} (Qtd atual: {Cores.VERMELHO}{medicamento_obj.quantidade}{Cores.RESET})")
        else:
            print(f"      {id_medicamento}. {Cores.AMARELO}{medicamento_obj.nome}{Cores.RESET} (Qtd atual: {Cores.MAGENTA}{medicamento_obj.quantidade}{Cores.RESET})")
    print(f"{Cores.BOLD}{Cores.AZUL}---------------------------------------------{Cores.RESET}")

    while True:
        try:
            print(f"Digite o {Cores.BOLD}ID{Cores.RESET} do medicamento para adicionar quantidade (ou '{Cores.AMARELO}0{Cores.RESET}' para cancelar): ", end="")
            id_escolhido_str = input().strip()
            if id_escolhido_str == '0':
                print(f"{Cores.AZUL}Operação de adição de quantidade cancelada.{Cores.RESET}")
                return

            id_escolhido = int(id_escolhido_str)
            if id_escolhido in estoque:
                medicamento_para_atualizar = estoque[id_escolhido]
                print(f"\nVocê escolheu: {medicamento_para_atualizar.nome} (Qtd atual: {medicamento_para_atualizar.quantidade})")
                while True:
                    try:
                        print(f"Digite a quantidade DE {Cores.AMARELO}{medicamento_para_atualizar.nome}{Cores.RESET} para ser SOMADA ao estoque: ", end="")
                        qtd_a_adicionar_str = input().strip()
                        qtd_a_adicionar = int(qtd_a_adicionar_str)
                        if qtd_a_adicionar >= 0:
                            medicamento_para_atualizar.quantidade += qtd_a_adicionar
                            print(f"{Cores.VERDE}Quantidade de '{medicamento_para_atualizar.nome}' atualizada para {medicamento_para_atualizar.quantidade} unidades.{Cores.RESET}")
                            return
                        else:
                            print(f"{Cores.AMARELO}A quantidade a ser somada não pode ser negativa. Tente novamente.{Cores.RESET}")
                    except ValueError:
                        print(f"{Cores.VERMELHO}Quantidade inválida. Por favor, digite um número inteiro.{Cores.RESET}")
            else:
                print(f"{Cores.AMARELO}ID '{id_escolhido_str}' não encontrado no estoque. Por favor, digite um ID existente.{Cores.RESET}")
        except ValueError:
            print(f"{Cores.VERMELHO}Entrada inválida para o ID. Por favor, digite um número.{Cores.RESET}")

def deletar_medicamento():
    """
    Permite ao usuário deletar uma entrada de medicamento do estoque,
    escolhendo-a por ID.
    """
    print(f"\n{Cores.BOLD}{Cores.VERMELHO}--- DELETAR MEDICAMENTO POR ID ---{Cores.RESET}")
    if not estoque:
        print(f"{Cores.AMARELO}Estoque vazio. Não há medicamentos para deletar.{Cores.RESET}")
        return

    print("Medicamentos disponíveis para deleção:")
    for id_medicamento, medicamento_obj in estoque.items():
        print(f"      {id_medicamento}. {Cores.AMARELO}{medicamento_obj.nome}{Cores.RESET} (Qtd: {medicamento_obj.quantidade})")
    print(f"{Cores.BOLD}{Cores.VERMELHO}----------------------------------{Cores.RESET}")

    while True:
        try:
            print(f"Digite o {Cores.BOLD}ID{Cores.RESET} do medicamento a ser deletado (ou '{Cores.AMARELO}0{Cores.RESET}' para cancelar): ", end="")
            id_deletar_str = input().strip()
            if id_deletar_str == '0':
                print(f"{Cores.AZUL}Operação de deleção de medicamento cancelada.{Cores.RESET}")
                return

            id_deletar = int(id_deletar_str)
            if id_deletar in estoque:
                nome_medicamento_deletado = estoque[id_deletar].nome
                del estoque[id_deletar]
                print(f"{Cores.VERDE}Medicamento '{nome_medicamento_deletado}' (ID: {id_deletar}) removido com sucesso do estoque.{Cores.RESET}")
                break
            else:
                print(f"{Cores.AMARELO}ID '{id_deletar_str}' não encontrado no estoque. Por favor, digite um ID existente.{Cores.RESET}")
        except ValueError:
            print(f"{Cores.VERMELHO}Entrada inválida para o ID. Por favor, digite um número inteiro.{Cores.RESET}")

### 4. Funções de Processamento de Pedidos e Resumo do Estoque
Estas funções permitem o processamento de pedidos de clientes e a visualização de um resumo consolidado do estoque. A função processar_pedidos simula uma venda, permitindo adicionar itens, verificar a necessidade de receita, aplicar descontos e atualizar o estoque. A função exibir_resumo fornece estatísticas gerais sobre o inventário.



In [15]:
def processar_pedidos():
    """
    Simula o processo de venda de medicamentos, permitindo adicionar itens
    ao pedido, verificar receita, aplicar desconto e atualizar o estoque.
    """
    print(f"\n{Cores.BOLD}{Cores.CIANO}--- PROCESSAR PEDIDOS ---{Cores.RESET}")
    if not estoque:
        print(f"{Cores.AMARELO}Estoque vazio. Não é possível processar pedidos.{Cores.RESET}")
        return

    pedido_atual = []
    continuar_adicionando_itens = True

    while continuar_adicionando_itens:
        print(f"\n{Cores.BOLD}{Cores.AZUL}--- Medicamentos Disponíveis ---{Cores.RESET}")
        print(f"  {'ID':<4} | {'Nome do Medicamento':<30} | {'Preço':<10} | {'Qtd':<3} | {'Info':<15}")
        print(f"  {'-'*4} | {'-'*30} | {'-'*10} | {'-'*3} | {'-'*15}")

        for id_med, med_obj in estoque.items():
            cor_qtd = Cores.VERMELHO if med_obj.quantidade <= LIMITE_ESTOQUE_CRITICO else Cores.VERDE
            id_field = f"{Cores.BRANCO}{id_med:<4}{Cores.RESET}"
            nome_field = f"{Cores.AMARELO}{med_obj.nome:<30.30}{Cores.RESET}"
            preco_text_val = f"R$ {med_obj.preco:.2f}"
            preco_field = f"{preco_text_val:>10}"
            qtd_numero_str = f"{med_obj.quantidade:>3}"
            qtd_colorido_str = f"{cor_qtd}{qtd_numero_str}{Cores.RESET}"
            qtd_field = qtd_colorido_str
            receita_text_display = "Req. Receita" if med_obj.receita else ""
            info_text_padded = f"{receita_text_display:<15.15}"
            info_field = f"{Cores.MAGENTA}{info_text_padded}{Cores.RESET}"
            print(f"  {id_field} | {nome_field} | {preco_field} | {qtd_field} | {info_field}")
        print(f"{Cores.BOLD}{Cores.AZUL}{'-'*76}{Cores.RESET}")

        id_escolhido_str = input(f"Digite o ID do medicamento para adicionar ao pedido (ou '0' para finalizar): ").strip()
        if id_escolhido_str == '0':
            continuar_adicionando_itens = False
            continue

        try:
            id_escolhido = int(id_escolhido_str)
            if id_escolhido not in estoque:
                print(f"{Cores.VERMELHO}ID inválido ou não encontrado. Tente novamente.{Cores.RESET}")
                continue

            medicamento_selecionado = estoque[id_escolhido]

            if medicamento_selecionado.quantidade == 0:
                print(f"{Cores.VERMELHO}O medicamento '{medicamento_selecionado.nome}' está esgotado.{Cores.RESET}")
                continue

            if medicamento_selecionado.receita:
                print(f"{Cores.MAGENTA}Atenção: O medicamento '{Cores.AMARELO}{medicamento_selecionado.nome}{Cores.RESET}' {Cores.MAGENTA}exige receita médica.{Cores.RESET}")
                while True:
                    confirmacao_receita = input("O cliente apresentou a receita? (s/n): ").lower().strip()
                    if confirmacao_receita in ['s', 'n']:
                        break
                    print(f"{Cores.VERMELHO}Resposta inválida. Por favor, digite 's' ou 'n'.{Cores.RESET}")
                if confirmacao_receita == 'n':
                    print(f"{Cores.AMARELO}Venda de '{medicamento_selecionado.nome}' não pode ser processada sem a apresentação da receita.{Cores.RESET}")
                    continue

            while True:
                try:
                    print(f"Digite a quantidade desejada de {Cores.AMARELO}{medicamento_selecionado.nome}{Cores.RESET} (Disponível: {medicamento_selecionado.quantidade}): ", end="")
                    qtd_desejada_str = input().strip()
                    qtd_desejada = int(qtd_desejada_str)

                    if qtd_desejada <= 0:
                        print(f"{Cores.VERMELHO}A quantidade deve ser um número positivo.{Cores.RESET}")
                        continue

                    quantidade_ja_no_carrinho = 0
                    item_existente_no_carrinho = None
                    for item_carrinho in pedido_atual:
                        if item_carrinho['id'] == id_escolhido:
                            item_existente_no_carrinho = item_carrinho
                            quantidade_ja_no_carrinho = item_carrinho['quantidade_pedida']
                            break
                    
                    if (quantidade_ja_no_carrinho + qtd_desejada) > medicamento_selecionado.quantidade:
                        disponivel_para_adicionar = medicamento_selecionado.quantidade - quantidade_ja_no_carrinho
                        print(f"{Cores.VERMELHO}Erro: Estoque insuficiente para adicionar {qtd_desejada} unidade(s).{Cores.RESET}")
                        print(f"  Você já tem {quantidade_ja_no_carrinho} no carrinho. Estoque total: {medicamento_selecionado.quantidade}.")
                        if disponivel_para_adicionar > 0:
                                print(f"  Você pode adicionar no máximo mais {Cores.VERDE}{disponivel_para_adicionar}{Cores.RESET} unidade(s).")
                        else:
                            print(f"  {Cores.AMARELO}Não há mais unidades disponíveis para adicionar deste item ao carrinho.{Cores.RESET}")
                        continue
                    
                    if item_existente_no_carrinho:
                        item_existente_no_carrinho['quantidade_pedida'] += qtd_desejada
                        print(f"{Cores.VERDE}Quantidade atualizada para '{medicamento_selecionado.nome}' no pedido: {item_existente_no_carrinho['quantidade_pedida']} unidade(s).{Cores.RESET}")
                    else:
                        pedido_atual.append({
                            'id': id_escolhido,
                            'nome': medicamento_selecionado.nome,
                            'quantidade_pedida': qtd_desejada,
                            'preco_unitario': medicamento_selecionado.preco
                        })
                        print(f"{Cores.VERDE}{qtd_desejada} unidade(s) de '{medicamento_selecionado.nome}' adicionada(s) ao pedido.{Cores.RESET}")
                    break
                except ValueError:
                    print(f"{Cores.VERMELHO}Quantidade inválida. Por favor, digite um número inteiro.{Cores.RESET}")
        except ValueError:
            print(f"{Cores.VERMELHO}ID inválido. Por favor, digite um número.{Cores.RESET}")
        except Exception as e:
            print(f"{Cores.VERMELHO}Ocorreu um erro inesperado: {e}{Cores.RESET}")

    if not pedido_atual:
        print(f"{Cores.AMARELO}Nenhum item no pedido. Processo de venda cancelado.{Cores.RESET}")
        return

    print(f"\n{Cores.BOLD}{Cores.VERDE}--- RESUMO DO PEDIDO ---{Cores.RESET}")
    valor_total_bruto = 0.0
    print(f"  {'Item':<5} | {'Nome do Medicamento':<30.30} | {'Qtd':>4} | {'Preço Unit.':>12} | {'Subtotal':>10}")
    print(f"  {'-'*5} | {'-'*30} | {'-'*4} | {'-'*12} | {'-'*10}")

    for i, item in enumerate(pedido_atual):
        subtotal_item = item['preco_unitario'] * item['quantidade_pedida']
        item_num_f = f"{i+1:<5}"
        nome_f = f"{item['nome']:<30.30}"
        qtd_f = f"{item['quantidade_pedida']:>4}"
        preco_unit_text = f"R$ {item['preco_unitario']:.2f}"
        preco_unit_f = f"{preco_unit_text:>12}"
        subtotal_text = f"R$ {subtotal_item:.2f}"
        subtotal_f = f"{subtotal_text:>10}"
        print(f"  {item_num_f} | {nome_f} | {qtd_f} | {preco_unit_f} | {subtotal_f}")
        valor_total_bruto += subtotal_item
    
    print(f"{Cores.BOLD}{'-'*75}{Cores.RESET}")
    print(f"{Cores.BOLD}Valor Total Bruto: R${valor_total_bruto:.2f}{Cores.RESET}")

    desconto_aplicado = 0.0
    cpf_cliente = None
    valor_final_a_pagar = valor_total_bruto

    while True:
        informar_cpf = input("Deseja informar o CPF para obter 20% de desconto? (s/n): ").lower().strip()
        if informar_cpf == 's':
            cpf_cliente = input("Digite o CPF do cliente: ").strip()
            if cpf_cliente:
                desconto_aplicado = valor_total_bruto * 0.20
                valor_final_a_pagar = valor_total_bruto - desconto_aplicado
                print(f"{Cores.VERDE}Desconto de 20% (R${desconto_aplicado:.2f}) aplicado.{Cores.RESET}")
            else:
                print(f"{Cores.AMARELO}CPF não informado. Nenhum desconto será aplicado.{Cores.RESET}")
                cpf_cliente = None
            break
        elif informar_cpf == 'n':
            print(f"{Cores.AMARELO}Nenhum desconto aplicado.{Cores.RESET}")
            break
        else:
            print(f"{Cores.VERMELHO}Opção inválida. Por favor, digite 's' ou 'n'.{Cores.RESET}")

    print(f"{Cores.BOLD}{Cores.VERDE}Valor Final a Pagar: R${valor_final_a_pagar:.2f}{Cores.RESET}")

    while True:
        print(f"Confirmar venda e atualizar estoque ({Cores.BOLD}{Cores.VERDE}s{Cores.RESET}/{Cores.BOLD}{Cores.VERMELHO}n{Cores.RESET})? ", end="")
        confirmar_venda = input().lower().strip()
        if confirmar_venda == 's':
            for item in pedido_atual:
                estoque[item['id']].quantidade -= item['quantidade_pedida']
            print(f"{Cores.VERDE}{Cores.BOLD}Venda processada com sucesso! Estoque atualizado.{Cores.RESET}")
            if cpf_cliente:
                print(f"CPF do cliente registrado: {cpf_cliente}")
            break
        elif confirmar_venda == 'n':
            print(f"{Cores.AMARELO}Venda cancelada pelo operador. Estoque não foi alterado.{Cores.RESET}")
            break
        else:
            print(f"{Cores.VERMELHO}Opção inválida. Digite 's' ou 'n'.{Cores.RESET}")

def exibir_resumo():
    """Exibe um resumo do estoque, incluindo totais e itens críticos."""
    print(f"\n{Cores.BOLD}{Cores.MAGENTA}--- RESUMO DO ESTOQUE ---{Cores.RESET}")
    if not estoque:
        print(f"{Cores.AMARELO}Estoque vazio.{Cores.RESET}")
        return

    total_medicamentos = len(estoque)
    total_unidades = 0
    valor_total_estoque = 0.0
    itens_em_critico = 0
    nomes_criticos = []

    for med_id, med_obj in estoque.items():
        total_unidades += med_obj.quantidade
        valor_total_estoque += med_obj.quantidade * med_obj.preco
        if med_obj.quantidade <= LIMITE_ESTOQUE_CRITICO:
            itens_em_critico +=1
            nomes_criticos.append(f"{med_obj.nome} (Qtd: {med_obj.quantidade})")
    
    print(f"Total de Tipos de Medicamentos (IDs diferentes): {Cores.CIANO}{total_medicamentos}{Cores.RESET}")
    print(f"Total de Unidades de Medicamentos no Estoque: {Cores.CIANO}{total_unidades}{Cores.RESET}")
    print(f"Valor Total Estimado do Estoque: {Cores.VERDE}R${valor_total_estoque:.2f}{Cores.RESET}")
    
    if itens_em_critico > 0:
        print(f"Medicamentos em Estoque Crítico ou Esgotado ({Cores.VERMELHO}{itens_em_critico}{Cores.RESET} tipo(s)):")
        for nome_crit in nomes_criticos:
            print(f"  - {Cores.AMARELO}{nome_crit}{Cores.RESET}")
    else:
        print(f"{Cores.VERDE}Nenhum medicamento em estoque crítico.{Cores.RESET}")
    print(f"{Cores.BOLD}{Cores.MAGENTA}--------------------------{Cores.RESET}")

# 5. Menu Principal e Execução do Programa
Este bloco final contém as funções responsáveis pela interface do usuário (menu), pela verificação de estoque crítico (estoque_critico) e pela função principal (main) que controla o loop de execução do programa e o direcionamento para as funcionalidades escolhidas. O script também inicializa o estoque com alguns medicamentos de exemplo quando executado.



In [17]:
def menu():
    print(f"\n{Cores.BOLD}{Cores.CIANO}--- MENU DE OPÇÕES ---{Cores.RESET}")
    tem_critico, qtd_criticos = estoque_critico()
    if tem_critico:
        print(f"{Cores.BOLD}{Cores.VERMELHO}AVISO: {qtd_criticos} ITENS EM ESTOQUE CRÍTICO/ESGOTADO!{Cores.RESET}")
        print(f"{Cores.BOLD}{Cores.VERMELHO}1. Listar Estoque Completo{Cores.RESET}")
    else:
        print(f"{Cores.AZUL}1. Listar Estoque Completo{Cores.RESET}")
    
    print(f"{Cores.AZUL}2. Adicionar Novo Medicamento{Cores.RESET}")
    print(f"{Cores.AZUL}3. Atualizar Quantidade por ID{Cores.RESET}")
    print(f"{Cores.AZUL}4. Deletar Entrada de Medicamento por ID{Cores.RESET}")
    print(f"{Cores.AZUL}5. Processar Pedidos por ID{Cores.RESET}")
    print(f"{Cores.AZUL}6. Exibir Resumo do Estoque{Cores.RESET}")
    print(f"{Cores.BOLD}{Cores.AMARELO}7. Sair{Cores.RESET}")
    print(f"{Cores.BOLD}{Cores.CIANO}---------------------------------------------{Cores.RESET}")

def estoque_critico():
    """Verifica se há itens em estoque crítico ou esgotados."""
    if not estoque:
        return False, 0

    itens_criticos = 0
    for medicamento_obj in estoque.values():
        if medicamento_obj.quantidade <= LIMITE_ESTOQUE_CRITICO:
            itens_criticos += 1
    return itens_criticos > 0, itens_criticos

def main():
    """Função principal que executa o loop do menu e interações."""
    while True:
        menu()
        opcao = input("Escolha uma opção: ").strip()

        if opcao == '1':
            listar_estoque()
        elif opcao == '2':
            adicionar_medicamento()
        elif opcao == '3':
            atualizar_estoque()
        elif opcao == '4':
            deletar_medicamento()
        elif opcao == '5':
            processar_pedidos()
        elif opcao == '6':
            exibir_resumo()
        elif opcao == '7':
            print(f"{Cores.BOLD}{Cores.AMARELO}Saindo do programa. Obrigado!{Cores.RESET}")
            break
        else:
            print(f"{Cores.VERMELHO}Opção inválida. Por favor, escolha uma opção de 1 a 7.{Cores.RESET}")

if __name__ == "__main__":
    # Adicionando medicamentos iniciais para demonstração
    med1 = Medicamento("Paracetamol 500mg", 12.50, False, True, 50, proximo_id_medicamento)
    estoque[proximo_id_medicamento] = med1
    proximo_id_medicamento += 1

    med5 = Medicamento("Clonazepam 2mg", 32.00, True, False, 0, proximo_id_medicamento) # Estoque esgotado
    estoque[proximo_id_medicamento] = med5
    proximo_id_medicamento += 1

    med2 = Medicamento("Dipirona Gotas 20ml", 11.90, False, True, 25, proximo_id_medicamento)
    estoque[proximo_id_medicamento] = med2
    proximo_id_medicamento += 1

    med3 = Medicamento("Amoxicilina 875mg", 45.99, True, False, 10, proximo_id_medicamento)
    estoque[proximo_id_medicamento] = med3
    proximo_id_medicamento += 1
    
    med4 = Medicamento("Ibuprofeno 400mg", 8.75, False, True, 5, proximo_id_medicamento) # Estoque crítico
    estoque[proximo_id_medicamento] = med4
    proximo_id_medicamento += 1

    main()


--- MENU DE OPÇÕES ---
AVISO: 2 ITENS EM ESTOQUE CRÍTICO/ESGOTADO!
1. Listar Estoque Completo
2. Adicionar Novo Medicamento
3. Atualizar Quantidade por ID
4. Deletar Entrada de Medicamento por ID
5. Processar Pedidos por ID
6. Exibir Resumo do Estoque
7. Sair
---------------------------------------------


Escolha uma opção:  6



--- RESUMO DO ESTOQUE ---
Total de Tipos de Medicamentos (IDs diferentes): 5
Total de Unidades de Medicamentos no Estoque: 90
Valor Total Estimado do Estoque: R$1426.15
Medicamentos em Estoque Crítico ou Esgotado (2 tipo(s)):
  - Clonazepam 2mg (Qtd: 0)
  - Ibuprofeno 400mg (Qtd: 5)
--------------------------

--- MENU DE OPÇÕES ---
AVISO: 2 ITENS EM ESTOQUE CRÍTICO/ESGOTADO!
1. Listar Estoque Completo
2. Adicionar Novo Medicamento
3. Atualizar Quantidade por ID
4. Deletar Entrada de Medicamento por ID
5. Processar Pedidos por ID
6. Exibir Resumo do Estoque
7. Sair
---------------------------------------------


Escolha uma opção:  1



--- LISTA COMPLETA DO ESTOQUE ---

1. PARACETAMOL 500MG
      Preço: R$12.50
      Quantidade: 50
      Exige Receita: Não
      É Genérico: Sim
      ID Único: 1
---------------------------------
2. CLONAZEPAM 2MG      ESGOTADO
      Preço: R$32.00
      Quantidade: 0
      Exige Receita: Sim
      É Genérico: Não
      ID Único: 2
---------------------------------
3. DIPIRONA GOTAS 20ML
      Preço: R$11.90
      Quantidade: 25
      Exige Receita: Não
      É Genérico: Sim
      ID Único: 3
---------------------------------
4. AMOXICILINA 875MG
      Preço: R$45.99
      Quantidade: 10
      Exige Receita: Sim
      É Genérico: Não
      ID Único: 4
---------------------------------
5. IBUPROFENO 400MG      ESTOQUE CRÍTICO
      Preço: R$8.75
      Quantidade: 5
      Exige Receita: Não
      É Genérico: Sim
      ID Único: 5
---------------------------------
--- FIM DA LISTA DE ESTOQUE ---

--- MENU DE OPÇÕES ---
AVISO: 2 ITENS EM ESTOQUE CRÍTICO/ESGOTADO!
1. Listar Estoque Comple

...

Desconto de 20% (R$57.12) aplicado.
Valor Final a Pagar: R$228.48
Confirmar venda e atualizar estoque (s/n)? 

 s


Venda processada com sucesso! Estoque atualizado.
CPF do cliente registrado: 13

--- MENU DE OPÇÕES ---
AVISO: 3 ITENS EM ESTOQUE CRÍTICO/ESGOTADO!
1. Listar Estoque Completo
2. Adicionar Novo Medicamento
3. Atualizar Quantidade por ID
4. Deletar Entrada de Medicamento por ID
5. Processar Pedidos por ID
6. Exibir Resumo do Estoque
7. Sair
---------------------------------------------


Escolha uma opção:  7


Saindo do programa. Obrigado!


# Conclusão sobre o Sistema de Gerenciamento de Estoque de Farmácia
Este projeto implementou com sucesso um sistema de console interativo e funcional para o gerenciamento de estoque de uma farmácia. Através de uma estrutura organizada em classes e funções, o programa oferece as operações essenciais de um CRUD (Criar, Ler, Atualizar, Deletar) para medicamentos, além de funcionalidades robustas para processamento de pedidos e visualização de resumos do inventário.

## Principais Destaques e Funcionalidades Alcançadas:

 - Interface de Usuário Amigável: O uso de um menu de console claro, enriquecido com cores, facilita a navegação e a interação do usuário.
 - Gestão Completa de Medicamentos: O sistema permite cadastrar novos medicamentos com detalhes importantes como preço, necessidade de receita, classificação como genérico, e quantidade inicial. A  - - -  - atualização de quantidade e a remoção de itens também são suportadas.
 - Controle de Estoque Inteligente: A implementação de IDs únicos para cada medicamento, juntamente com o aviso dinâmico de estoque crítico ou esgotado, auxilia na gestão eficiente do inventário.
 - Processamento de Pedidos Realista: A funcionalidade de processar pedidos inclui verificações importantes (disponibilidade, exigência de receita) e até mesmo um sistema de desconto por CPF, adicionando um toque de realismo.
 - Robustez e Validação: Diversas proteções contra erros de entrada do usuário foram implementadas, tornando o sistema mais resiliente a entradas inesperadas.
 - Organização do Código: A utilização da classe Medicamento para modelar os produtos e da classe Cores para a interface demonstra uma boa aplicação dos princípios de orientação a objetos, tornando o código mais legível e modular.
## Possíveis Melhorias e Próximos Passos:

Apesar de ser um sistema de console completo e bem elaborado, algumas evoluções poderiam ser consideradas para futuras versões:

Persistência de Dados: Implementar o salvamento e carregamento do estoque em um arquivo (CSV, JSON) ou um banco de dados simples (como SQLite) para que os dados não sejam perdidos ao fechar a aplicação.
Busca e Filtragem Avançada: Adicionar opções para buscar medicamentos por nome, categoria, ou filtrar por genéricos, medicamentos que exigem receita, etc.
Relatórios Detalhados: Gerar relatórios de vendas por período, medicamentos mais vendidos, ou histórico de movimentação de estoque.
Interface Gráfica (GUI): Para uma experiência de usuário mais rica, o sistema poderia ser migrado para uma interface gráfica utilizando bibliotecas como Tkinter, PyQt, ou Kivy.
Controle de Usuários: Adicionar diferentes níveis de acesso (ex: administrador, vendedor) com login e senha.
Validação de CPF (Opcional): Se o objetivo for maior rigor, implementar uma lógica de validação real para o formato do CPF.
Em suma, o projeto atual demonstra uma excelente compreensão dos conceitos de programação procedural e orientada a objetos, resultando em uma aplicação de console funcional, interativa e com várias funcionalidades úteis para o contexto de uma farmácia. É uma base sólida que pode ser expandida e aprimorada com novos recursos.

